In [1]:
# import via notebook
!mongoimport --type json -d wine -c ratings --drop --jsonArray ../Resources/clean_wine_data_final.json

2024-05-07T19:58:32.716-0700	connected to: mongodb://localhost/
2024-05-07T19:58:32.716-0700	dropping: wine.ratings
2024-05-07T19:58:34.831-0700	77931 document(s) imported successfully. 0 document(s) failed to import.


In [2]:
# Importing Necessary Libraries
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import tensorflow as tf

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
mongo.list_database_names()

['admin',
 'air_b_and_b',
 'autosaurus',
 'classDB',
 'config',
 'gardenDB',
 'local',
 'met',
 'uk_food',
 'wine']

In [5]:
db = mongo['wine']
db.list_collection_names()

['ratings']

In [6]:
#assign the collection to a variable
wine_df = db['ratings']

In [7]:
cursor = db["ratings"].find({})
json_data = list(cursor)

In [8]:
wine_df = pd.DataFrame(json_data)
wine_df.head()

,_id,country,description,points,price,province,region,title,variety,winery,rating_category,type,vintage
0,663aea58dbf22398a34f3758,Argentina,"Baked plum, molasses, balsamic vinegar and che...",87,30,Other,Cafayate,Felix Lavaque 2010 Felix Malbec (Cafayate),Malbec,Felix Lavaque,Good,Red,2010
1,663aea58dbf22398a34f3759,US,Building on 150 years and six generations of w...,87,12,California,Central Coast,Mirassou 2012 Chardonnay (Central Coast),Chardonnay,Mirassou,Good,White,2012
2,663aea58dbf22398a34f375a,US,"Pineapple rind, lemon pith and orange blossom ...",87,13,Michigan,Lake Michigan Shore,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Good,White,2013
3,663aea58dbf22398a34f375b,US,"Soft, supple plum envelopes an oaky structure ...",87,19,California,Napa Valley,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,Good,Red,2011
4,663aea58dbf22398a34f375c,Argentina,Raw black-cherry aromas are direct and simple ...,87,13,Mendoza Province,Mendoza,Gaucho Andino 2011 Winemaker Selection Malbec ...,Malbec,Gaucho Andino,Good,Red,2011


In [9]:
# Drop id columns
wine_df.drop(columns=['_id', 'description', 'title', 'winery'], inplace=True)

In [10]:
wine_df

,country,points,price,province,region,variety,rating_category,type,vintage
0,Argentina,87,30,Other,Cafayate,Malbec,Good,Red,2010
1,US,87,12,California,Central Coast,Chardonnay,Good,White,2012
2,US,87,13,Michigan,Lake Michigan Shore,Riesling,Good,White,2013
3,US,87,19,California,Napa Valley,Cabernet Sauvignon,Good,Red,2011
4,Argentina,87,13,Mendoza Province,Mendoza,Malbec,Good,Red,2011
...,...,...,...,...,...,...,...,...,...
77926,US,91,55,California,Sonoma Coast,Pinot Noir,Excellent,Red,2014
77927,US,90,35,California,Napa Valley,Zinfandel,Very Good,Red,2012
77928,France,90,28,Alsace,Alsace,Pinot Gris,Very Good,White,2013
77929,US,90,75,Oregon,Oregon,Pinot Noir,Very Good,Red,2004


Examine columns for encoding

In [11]:
wine_df.nunique()

country               7
points               21
price               341
province             63
region             1000
variety              21
rating_category       6
type                  4
vintage              40
dtype: int64

In [12]:
wine_df['province'].value_counts()

province
California            31456
Washington             7387
Oregon                 4821
Tuscany                4225
Bordeaux               3390
                      ...  
Northwestern Italy        2
Iowa                      2
Kentucky                  1
Rhode Island              1
Hawaii                    1
Name: count, Length: 63, dtype: int64

In [13]:
wine_df['province'].value_counts()[wine_df['province'].value_counts() > 1]

province
California              31456
Washington               7387
Oregon                   4821
Tuscany                  4225
Bordeaux                 3390
Burgundy                 3058
Mendoza Province         2731
Piedmont                 2292
New York                 2252
Northern Spain           1770
Alsace                   1446
Champagne                1361
Loire Valley             1085
Provence                 1011
Catalonia                 981
Southwest France          872
Rhône Valley              744
Veneto                    666
Sicily & Sardinia         648
South Australia           625
Virginia                  545
Languedoc-Roussillon      481
Northeastern Italy        470
France Other              448
Central Italy             412
Other                     356
Central Spain             338
Lombardy                  302
Southern Italy            208
Levante                   201
Western Australia         200
Victoria                  175
Australia Other           151
B

In [14]:
# columns to drop: region, province
wine_df.drop(columns=['region', 'province'], inplace=True)

# columns for encoding: country, variety, rating catagory and type

In [15]:
wine_df['vintage'] = wine_df['vintage'].replace('No Year', pd.NA)
wine_df.dropna()

,country,points,price,variety,rating_category,type,vintage
0,Argentina,87,30,Malbec,Good,Red,2010
1,US,87,12,Chardonnay,Good,White,2012
2,US,87,13,Riesling,Good,White,2013
3,US,87,19,Cabernet Sauvignon,Good,Red,2011
4,Argentina,87,13,Malbec,Good,Red,2011
...,...,...,...,...,...,...,...
77926,US,91,55,Pinot Noir,Excellent,Red,2014
77927,US,90,35,Zinfandel,Very Good,Red,2012
77928,France,90,28,Pinot Gris,Very Good,White,2013
77929,US,90,75,Pinot Noir,Very Good,Red,2004


In [16]:
wine_df.head()

,country,points,price,variety,rating_category,type,vintage
0,Argentina,87,30,Malbec,Good,Red,2010
1,US,87,12,Chardonnay,Good,White,2012
2,US,87,13,Riesling,Good,White,2013
3,US,87,19,Cabernet Sauvignon,Good,Red,2011
4,Argentina,87,13,Malbec,Good,Red,2011


In [17]:
#country
countries = []
country_encoding_count = []
counter = 0
for unique in wine_df['country'].unique():
    countries.append(unique)
for country in countries:
    counter += 1
    country_encoding_count.append(counter)

print(countries)
print(country_encoding_count)

['Argentina', 'US', 'Italy', 'France', 'Australia', 'Spain', 'Canada']
[1, 2, 3, 4, 5, 6, 7]


In [18]:
country_dict = dict(zip(countries, country_encoding_count))
country_dict

{'Argentina': 1,
 'US': 2,
 'Italy': 3,
 'France': 4,
 'Australia': 5,
 'Spain': 6,
 'Canada': 7}

In [19]:
wine_df['country'] = wine_df['country'].map(country_dict)
wine_df.head()

,country,points,price,variety,rating_category,type,vintage
0,1,87,30,Malbec,Good,Red,2010
1,2,87,12,Chardonnay,Good,White,2012
2,2,87,13,Riesling,Good,White,2013
3,2,87,19,Cabernet Sauvignon,Good,Red,2011
4,1,87,13,Malbec,Good,Red,2011


In [20]:
# variety
variety = []
variety_encoding_count = []
counter = 0
for unique in wine_df['variety'].unique():
    variety.append(unique)
for i in variety:
    counter += 1
    variety_encoding_count.append(counter)
variety_dict = dict(zip(variety, variety_encoding_count))
wine_df['variety'] = wine_df['variety'].map(variety_dict)
wine_df.head()

,country,points,price,variety,rating_category,type,vintage
0,1,87,30,1,Good,Red,2010
1,2,87,12,2,Good,White,2012
2,2,87,13,3,Good,White,2013
3,2,87,19,4,Good,Red,2011
4,1,87,13,1,Good,Red,2011


In [21]:
variety_dict


{'Malbec': 1,
 'Chardonnay': 2,
 'Riesling': 3,
 'Cabernet Sauvignon': 4,
 'Pinot Noir': 5,
 'White Blend': 6,
 'Red Blend': 7,
 'Merlot': 8,
 'Pinot Gris': 9,
 'Sauvignon Blanc': 10,
 'Sangiovese': 11,
 'Cabernet Franc': 12,
 'Champagne Blend': 13,
 'Rosé': 14,
 'Zinfandel': 15,
 'Bordeaux-style Red Blend': 16,
 'Syrah': 17,
 'Nebbiolo': 18,
 'Rhône-style Red Blend': 19,
 'Sparkling Blend': 20,
 'Tempranillo': 21}

In [22]:
# rating_category_dict
rating_category = []
rating_category_encoding_count = []
counter = 0
for unique in wine_df['rating_category'].unique():
    rating_category.append(unique)
for i in rating_category:
    counter += 1
    rating_category_encoding_count.append(counter)
rating_category_dict = dict(zip(rating_category, rating_category_encoding_count))
wine_df['rating_category'] = wine_df['rating_category'].map(rating_category_dict)
wine_df.head()

,country,points,price,variety,rating_category,type,vintage
0,1,87,30,1,1,Red,2010
1,2,87,12,2,1,White,2012
2,2,87,13,3,1,White,2013
3,2,87,19,4,1,Red,2011
4,1,87,13,1,1,Red,2011


In [23]:
rating_category_dict

{'Good': 1,
 'Very Good': 2,
 'Excellent': 3,
 'Acceptable': 4,
 'Superb': 5,
 'Classic': 6}

In [24]:
# type
type = []
type_encoding_count = []
counter = 0
for unique in wine_df['type'].unique():
    type.append(unique)
for i in type:
    counter += 1
    type_encoding_count.append(counter)
type_category_dict = dict(zip(type, type_encoding_count))
wine_df['type'] = wine_df['type'].map(type_category_dict)
wine_df.head()

,country,points,price,variety,rating_category,type,vintage
0,1,87,30,1,1,1,2010
1,2,87,12,2,1,2,2012
2,2,87,13,3,1,2,2013
3,2,87,19,4,1,1,2011
4,1,87,13,1,1,1,2011


In [25]:
type_category_dict

{'Red': 1, 'White': 2, 'Sparkling': 3, 'Rosé': 4}

In [26]:
# rating_category_dict
vintage = []
vintage_encoding_count = []
counter = 0
for unique in wine_df['vintage'].unique():
    vintage.append(unique)
for i in vintage:
    counter += 1
    vintage_encoding_count.append(counter)
vintage_category_dict = dict(zip(vintage, vintage_encoding_count))
wine_df['vintage'] = wine_df['vintage'].map(vintage_category_dict)
wine_df.head()

,country,points,price,variety,rating_category,type,vintage
0,1,87,30,1,1,1,1
1,2,87,12,2,1,2,2
2,2,87,13,3,1,2,3
3,2,87,19,4,1,1,4
4,1,87,13,1,1,1,4


In [27]:
# # Split our preprocessed data into our features and target arrays
# y = wine_dummies_df['']
# X = wine_dummies_df.drop (columns = '')

# # Split the preprocessed data into a training and testing dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [28]:
# # Create a StandardScaler instances
# scaler = StandardScaler()

# # Fit the StandardScaler
# X_scaler = scaler.fit(X_train)

# # Scale the data
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)